##  $ NOMBRE: ANGIE\ ESCOBEDO\ MESCCO $

In [5]:
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 204.2 MB 33 kB/s 
     |████████████████████████████████| 198 kB 28.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=39a221697f5fa734a04252fc2fc255d0de02c60efa394e78e352d2a482777ec8
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark


In [7]:
from pyspark import SparkContext
sc = SparkContext()

## *1. BAGOFWORDS*

In [58]:
def bag_of_word(Corpus):
  
  x = Corpus.strip().lower().split()
  return list(filter(lambda x: len(x)>2,x)) # solo palabras mayor a dos de tamaño

In [62]:
CORPUS = ["most other parts of the world at no cost and future with almost no restrictions",
          "have to check the country laws of the country where are located before using",
          "most you are not located in the United States future"]

Corpus_words = sc.parallelize(CORPUS,6)
Corpus_words = Corpus_words.map(bag_of_word)
Corpus_words.take(3)

[['most',
  'other',
  'parts',
  'the',
  'world',
  'cost',
  'and',
  'future',
  'with',
  'almost',
  'restrictions'],
 ['have',
  'check',
  'the',
  'country',
  'laws',
  'the',
  'country',
  'where',
  'are',
  'located',
  'before',
  'using'],
 ['most', 'you', 'are', 'not', 'located', 'the', 'united', 'states', 'future']]

## *2.TF IMPLEMENTADO*

In [63]:
import math
def TF_Implementado(Corpus):
    """Funcion que calcula el tf de un documento 

    Args:
        Doc (str): un Str que es un documento que contiene palabras

    Returns:
        (str,double): un RDD contiene (palabra,double) tupla."""
    
    # Calculamos words del corpus
    cant_words=len(Corpus)
    # Obtener lista sin las repeticiones del corpus
    cant_sin_duplicar=list(set(Corpus))

    #Obtenemos el tf para cada palabra
    mapeo=list(map(lambda x:(x,Corpus.count(x)),cant_sin_duplicar))
    
    tf=list(map(lambda x:(x[0],1+math.log10(x[1]/cant_words)),mapeo))# Calculamos logaritmicamente

    return (tf)

In [66]:
CORPUS = ["most other parts of the world at no cost and future with almost no restrictions",
          "have to check the country laws of the country where are located before using",
          "most you are not located in the United States future"]

Corpus_words = sc.parallelize(CORPUS,3)
Corpus_words = Corpus_words.map(bag_of_word)
Corpus_TF=Corpus_words.map(TF_Implementado)
Corpus_TF.take(2)

[[('cost', -0.04139268515822492),
  ('most', -0.04139268515822492),
  ('world', -0.04139268515822492),
  ('with', -0.04139268515822492),
  ('other', -0.04139268515822492),
  ('almost', -0.04139268515822492),
  ('the', -0.04139268515822492),
  ('restrictions', -0.04139268515822492),
  ('future', -0.04139268515822492),
  ('and', -0.04139268515822492),
  ('parts', -0.04139268515822492)],
 [('where', -0.07918124604762489),
  ('using', -0.07918124604762489),
  ('located', -0.07918124604762489),
  ('have', -0.07918124604762489),
  ('laws', -0.07918124604762489),
  ('before', -0.07918124604762489),
  ('are', -0.07918124604762489),
  ('the', 0.22184874961635637),
  ('check', -0.07918124604762489),
  ('country', 0.22184874961635637)]]

## *3. IDF IMPLEMENTACION*

In [70]:
import math
def IDF_Implementacion(Document,corpus):
    """Funcion que calcula el IDF de un corpus de TF """
    Documento_New=Document.flatMap(lambda x:x)
    Documento_New=Documento_New.map(lambda x:x[0]).distinct()
    #Total documentos en corpus
    total=corpus.count()
    corpus_collection=corpus.collect()
    #Extraer cantidad de veces de cada palabra los corpus
    IDF=Documento_New.map(lambda x:list(map(lambda y:(x,1 if x in y else 0),corpus_collection)))
    IDF_new=IDF.flatMap(lambda x:x)
    IDF_new=IDF_new.groupByKey().map(lambda x:(x[0],sum(x[1])))
    IDF_new=IDF_new.map(lambda x:(x[0],math.log10(1+total/x[1])))# Calculamos logaritmicamente
    return IDF_new

In [71]:
CORPUS = ["most other parts of the world at no cost and future with almost no restrictions",
          "have to check the country laws of the country where are located before using",
          "most you are not located in the United States future"]

Corpus_words = sc.parallelize(CORPUS,3)
Corpus_words = Corpus_words.map(bag_of_word)
Corpus_TF = Corpus_words.map(TF_Implementado)
Corpus_IDF = IDF_Implementacion(Corpus_TF,Corpus_words)
Corpus_IDF.collect()


[('cost', 0.6020599913279624),
 ('most', 0.3979400086720376),
 ('with', 0.6020599913279624),
 ('almost', 0.6020599913279624),
 ('the', 0.3010299956639812),
 ('parts', 0.6020599913279624),
 ('located', 0.3979400086720376),
 ('have', 0.6020599913279624),
 ('before', 0.6020599913279624),
 ('are', 0.3979400086720376),
 ('not', 0.6020599913279624),
 ('states', 0.6020599913279624),
 ('world', 0.6020599913279624),
 ('future', 0.3979400086720376),
 ('and', 0.6020599913279624),
 ('where', 0.6020599913279624),
 ('using', 0.6020599913279624),
 ('laws', 0.6020599913279624),
 ('united', 0.6020599913279624),
 ('you', 0.6020599913279624),
 ('other', 0.6020599913279624),
 ('restrictions', 0.6020599913279624),
 ('check', 0.6020599913279624),
 ('country', 0.6020599913279624)]

In [72]:
def RecIDF(word,corpusIDF):
        """Funcion que recuperar el valor IDF de una palabra"""
        L=corpusIDF
        EL=list(filter(lambda x:x[0]==word,L))
        return EL[0][1]

## *4.TF-IDF IMPLEMENTACION*

In [74]:
import math

def TF_IDF(TextoTF,TextoIDF):
    """Funcion que calcula el IDF para cada Doc de TF"""
    corpusIDF=TextoIDF.collect()
    TFIDF=TextoTF.map(lambda x:list(map((lambda y:(y[0],y[1]*RecIDF(y[0],corpusIDF))),x)))
    return  TFIDF


In [75]:
CORPUS = ["most other parts of the world at no cost and future with almost no restrictions",
          "have to check the country laws of the country where are located before using",
          "most you are not located in the United States future"]

Corpus_words = sc.parallelize(CORPUS,3)
Corpus_words = Corpus_words.map(bag_of_word)
Corpus_TF = Corpus_words.map(TF_Implementado)
Corpus_IDF = IDF_Implementacion(Corpus_TF,Corpus_words)
Corpus_TF_IDF = TF_IDF(Corpus_TF,Corpus_IDF)
Corpus_TF_IDF.collect()

[[('cost', -0.02492087966740197),
  ('most', -0.016471805490822947),
  ('world', -0.02492087966740197),
  ('with', -0.02492087966740197),
  ('other', -0.02492087966740197),
  ('almost', -0.02492087966740197),
  ('the', -0.012460439833700985),
  ('restrictions', -0.02492087966740197),
  ('future', -0.016471805490822947),
  ('and', -0.02492087966740197),
  ('parts', -0.02492087966740197)],
 [('where', -0.0476718603087703),
  ('using', -0.0476718603087703),
  ('located', -0.03150938573885459),
  ('have', -0.0476718603087703),
  ('laws', -0.0476718603087703),
  ('before', -0.0476718603087703),
  ('are', -0.03150938573885459),
  ('the', 0.06678312813507141),
  ('check', -0.0476718603087703),
  ('country', 0.13356625627014282)],
 [('located', 0.01820873619052574),
  ('most', 0.01820873619052574),
  ('united', 0.02754875437014939),
  ('not', 0.02754875437014939),
  ('you', 0.02754875437014939),
  ('the', 0.013774377185074694),
  ('are', 0.01820873619052574),
  ('states', 0.02754875437014939),